# Auto encoder

## Loading and transforming the data

We will consider the MNIST database which contains 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. in a first time, we load the data.

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,UpSampling2D,InputLayer,ReLU,BatchNormalization
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

def loadMNISTAsVector(subset=10000) :
    nb_classes=10
    (X_train_img, y_train_real), (X_test_img, y_test_real) = mnist.load_data()
    X_train_vect = X_train_img[:subset].reshape(subset, 784)
    X_test_vect = X_test_img.reshape(10000, 784)
    X_train_vect = X_train_vect.astype("float32")
    X_test_vect = X_test_vect.astype("float32")
    X_train_vect /= 255
    X_test_vect /= 255
    y_train_cat = to_categorical(y_train_real[:subset], nb_classes)
    y_test_cat = to_categorical(y_test_real, nb_classes)
    return (X_train_vect, y_train_cat), (X_test_vect, y_test_cat)
    
def loadMNISTAsMaxtrix(subset=10000) :
    nb_classes=10
    img_rows, img_cols = 28, 28
    (X_train_img, y_train_real), (X_test_img, y_test_real) = mnist.load_data()
    X_train_mat = X_train_img[:subset].reshape(X_train_img[:subset].shape[0], img_rows, img_cols, 1)
    X_test_mat = X_test_img.reshape(X_test_img.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    X_train_mat = X_train_mat.astype('float32')
    X_test_mat = X_test_mat.astype('float32')
    X_train_mat /= 255
    X_test_mat /= 255
    y_train_cat = to_categorical(y_train_real[:subset], nb_classes)
    y_test_cat = to_categorical(y_test_real, nb_classes)
    return (X_train_mat, y_train_cat), (X_test_mat, y_test_cat)

def compare(decoded_imgs,img):
    n = 10  # how many digits we will display
    plt.figure(figsize=(20, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(img[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

(X_train_mat, y_train_cat), (X_test_mat, y_test_cat) =loadMNISTAsMaxtrix()
(X_train_vect, y_train_cat), (X_test_vect, y_test_cat) =loadMNISTAsVector()



We now have vectors instead of images

## Multi layered auto encoder

In [ ]:
def getAutoEncoder(encoding_dim = 1):
    model = Sequential()
    #encoder
    model.add(InputLayer(input_shape=(784,)))
    
    #space reduction
    model.add(Dense(encoding_dim,activation='relu',name='featurespace'))
   
    #Decoder
     
    

    model.add(Dense(784, activation='sigmoid'))
    
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',  optimizer=sgd)

   
    return model

model=getAutoEncoder(encoding_dim = 1)
model.summary()





In [ ]:
(X_train_vect, _), (X_test_vect, _) =loadMNISTAsVector(subset=10000)
model=getAutoEncoder(encoding_dim = 32)
batch_size = 256
epochs=20
hist = model.fit(X_train_vect, X_train_vect,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test_vect, X_test_vect))

In [ ]:
decoded_imgs = model.predict(X_test_vect)
compare(decoded_imgs,X_test_vect )

# Dimension reduction

You can make a model that predict the values corresponding to the neurons inside an hidden layer by the following way :

```
layer_output = model.get_layer('featurespace').output
latent=Model(inputs=model.input,outputs=layer_output)
```
# Questions
* by using an encoding dimension of size 2, represent in 2 dimension the MNIST data with an auto encoder. Add the colors corresponding to the class. Use different activation function just before the hidden representation to see the difference
* build an auto encode for all the dataset except the images corresponding to 5. Build the latent representation of all the data and try to detect the 5 as an outlier.

## CNN auto encoders

In [6]:
def getCNNAutoEncoder(encoding_dim = 1):
    model = Sequential()
    #encoder
    model.add(InputLayer(input_shape=(28,28,1)))
    model.add(Conv2D(1, (3, 3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2))) # images 14x14
    model.add(Conv2D(1,(3, 3),  padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))# images 7x7
    
    
    
    model.add(Flatten())                    #32x7x7
    
    model.add(Dense(encoding_dim,activation='relu',name='featurespace')) #16
   
    
    nb_img=1
    model.add(Dense(nb_img*7*7,activation='relu')) #nb_imgx7x7
    #Decoder
    model.add(Reshape((7, 7,nb_img))) #nb_imgx7x7
    model.add(UpSampling2D((2, 2))) #images 14x14
    model.add(Conv2D(1, (3, 3), padding='same', activation='relu'))
   
    model.add(UpSampling2D((2, 2))) #images 28x28
   
    model.add(Conv2D(1, (3, 3),  padding='same', activation='sigmoid'))
    
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',  optimizer=Adam())

   
    return model


In [ ]:
model=getCNNAutoEncoder(encoding_dim = 32)
model.summary()

In [ ]:
(X_train_mat, _), (X_test_mat, _) =loadMNISTAsMaxtrix()
model=getCNNAutoEncoder(encoding_dim = 32)
batch_size = 256
epochs=50
model.fit(X_train_mat, X_train_mat,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test_mat, X_test_mat))

In [ ]:
decoded_imgs = model.predict(X_test_mat)
compare(decoded_imgs,X_test_mat )

# Denoising

In [9]:
noise_factor = 0.1
x_train_noisy = X_train_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train_mat.shape) 
x_test_noisy = X_test_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test_mat.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
decoded_imgs = model.predict(x_test_noisy)
compare(decoded_imgs,x_test_noisy )

# Image correction

## denoising

In [11]:
noise_factor = 0.5
x_train_noisy = X_train_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train_mat.shape) 
x_test_noisy = X_test_mat + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_test_mat.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
model=getCNNAutoEncoder(encoding_dim = 32)
batch_size = 256
epochs=50
model.fit(x_train_noisy, X_train_mat,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(x_test_noisy, X_test_mat))

In [ ]:
decoded_imgs = model.predict(x_test_noisy)
compare(decoded_imgs,x_test_noisy )

In [12]:
(X_train_mat, _), (X_test_mat, _) =loadMNISTAsMaxtrix()
X_train_crop=np.copy(X_train_mat)
X_train_crop[:,:14,:14,]=0
X_test_crop=np.copy(X_test_mat)
X_test_crop[:,:14,:14,]=0


In [ ]:
model=getCNNAutoEncoder(encoding_dim = 32)
batch_size = 256
epochs=50
model.fit(X_train_crop, X_train_mat,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test_crop, X_test_mat))

In [ ]:
decoded_imgs = model.predict(X_test_crop)
compare(decoded_imgs,X_test_crop )